In [14]:
#importing the SQlite3 librbary
import sqlite3 
import numpy as np

In [15]:
#Creating connection to SQLite DB
conn = sqlite3.connect('vector-db.db') #vector-db is the name of the database

In [16]:
#For checking the connection
conn

In [17]:
#create a cursor obeject to execute SQL commands
cursor = conn.cursor()

In [18]:
#create a table for vector data
cursor.execute(
"""
CREATE TABLE IF NOT EXISTS vectors (
    id INTEGER PRIMARY KEY,
    vector BLOB NOT NULL
)
"""
)
#BLOB is used because we are adding binary objects in the table vectors

In [20]:
#Generating some sample vectors
vect1 = np.array([1.2, 3.4, 2.1, 0.8])
vect2 = np.array([2.7, 1.5, 3.9, 2.3])

In [21]:
#Converting the data into binary
vect1.tobytes()

b'333333\xf3?333333\x0b@\xcd\xcc\xcc\xcc\xcc\xcc\x00@\x9a\x99\x99\x99\x99\x99\xe9?'

In [22]:
#Insert vector data into the table
cursor.execute(
"INSERT INTO vectors (vector) VALUES (?)",
    (sqlite3.Binary(vect1.tobytes()),)
)

In [23]:
#Insert vector data into the table
cursor.execute(
"INSERT INTO vectors (vector) VALUES (?)",
    (sqlite3.Binary(vect2.tobytes()),)
)

In [24]:
#Retrieve data
cursor.execute("SELECT vector FROM vectors")

In [25]:
#Brings all the rows to memory
rows = cursor.fetchall()

In [26]:
#checking the rows
rows

[(b'333333\xf3?333333\x0b@\xcd\xcc\xcc\xcc\xcc\xcc\x00@\x9a\x99\x99\x99\x99\x99\xe9?',),
 (b'\x9a\x99\x99\x99\x99\x99\x05@\x00\x00\x00\x00\x00\x00\xf8?333333\x0f@ffffff\x02@',)]

In [41]:
#converting the binary data to the original vector data for one row
vector = np.frombuffer(rows[0][0] , dtype=np.float64)

In [42]:
#printing the vector
vector

array([1.2, 3.4, 2.1, 0.8])

In [43]:
#converting the binary to the original vector data for all rows
vectors = []
for row in rows:
    vector = np.frombuffer(row[0] , dtype=np.float64)
    vectors.append(vector)

In [44]:
#printing the vectors
vectors

[array([1.2, 3.4, 2.1, 0.8]), array([2.7, 1.5, 3.9, 2.3])]

In [12]:
#For commiting/saving the changes to the DB
conn.commit()

In [13]:
#For closing the connection with the DB
conn.close()

# Vector Similarity Search

In [70]:
#Query vector
query_vect = np.array([1.0, 3.2, 2.0, 0.5])

In [71]:
#Quering to find the closest vector to the query vector
cursor.execute("""
SELECT vector FROM vectors ORDER BY abs(vector - ?) ASC
""", (sqlite3.Binary(query_vect.tobytes()),)
)

In [72]:
#finding the top one
result = cursor.fetchone()

In [73]:
#most similar vector (output is in binary)
result

(b'\x9a\x99\x99\x99\x99\x99\x05@\x00\x00\x00\x00\x00\x00\xf8?333333\x0f@ffffff\x02@',)

In [74]:
#most similar vector
np.frombuffer(result[0], dtype = np.float64)

array([2.7, 1.5, 3.9, 2.3])